In [1]:
import numpy as np
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *
import utils.pulse_configs as pc
import time
from exp_ctrl_vqe_cost_function import exp_g1_cost_func_xt
from scipy.optimize import minimize, basinhopping, brute

In [2]:
config_dict = pc.config_dict

In [3]:
# Defining relevant values
q_ch = config_dict["qubit"]["ch"] 
A_ch = config_dict["storage_A"]["ch"]
B_ch = config_dict["storage_B"]["ch"]

wq = config_dict["qubit"]["selective_freqs"]["g"]["ge"]
wa_A = config_dict["storage_A"]["freq"]
wa_B = config_dict["storage_B"]["freq"]

In [4]:
# ========== OPTIONS ========== #
max_segs = 20
time_start = 0.0000000
time_stop = 0.000002
init_qamp = 942
init_camp = 4465

num_drives = 2
chs = [2, 4]
init_freqs = [wq, wa_A]
verbose = True
plot_opt_pulses = False
plot_pulse_viewer = False
start_split_num = 10
n_seg_jump = 1

load_pulse_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\exp_prep_1g_interp_classical'
load_times_file = r'opt_SNAP_times_1g.txt'
load_amps_file = r'opt_SNAP_amps_1g.txt'
use_loaded_data = False

save_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\exp_prep_1g_interp_classical\save_data'
hdf5_name = time.strftime('%Y%m%d-%H%M%S.csv')
hdf5_file_path = os.path.join(save_dir, hdf5_name)

hf = h5py.File(hdf5_file_path, "w")
hf.close()

maxiter_start = 1
maxiter_step = 1
qubit_amp_scale = 4
cavity_amp_scale = 4
# ========== OPTIONS ========== #

drive_freqs = np.array(init_freqs)

qscale = []
cscale = []
for i in range(2):
    qscale.append((-init_qamp * qubit_amp_scale, init_qamp * qubit_amp_scale))
    cscale.append((-init_camp * cavity_amp_scale, init_camp * cavity_amp_scale))

t_segs, amp_segs = setup_interp_segs(2 * num_drives, time_start, time_stop, init_qamp)

amp_segs[:2, :] = init_qamp
amp_segs[2:, :] = init_camp

amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

t_segs = np.reshape(t_segs, (num_drives * 2, int(len(t_segs.flatten()) / (num_drives * 2))))

t_segs = t_segs[:, 1:-1]

for i in range(start_split_num - 1):
    t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))
    t_segs = t_segs[:, 1:-1]
    
t_segs = interp_time_wrapper(t_segs, time_start, time_stop)

# Flips occasional bits to give it a random 
flip_bits = np.array([[False, False, True, True]])
flip_mask = np.array([np.repeat(flip_bits, np.ceil(start_split_num / 4), axis=0).flatten()[:start_split_num]])
flip_mask = np.repeat([flip_mask], num_drives * 2, axis=1)[0]

amp_segs[flip_mask] *= -1

if use_loaded_data:
    print("Loading data")
    load_pulse_times = np.loadtxt(str(os.path.join(load_pulse_dir, load_times_file)))
    load_pulse_amps = np.loadtxt(str(os.path.join(load_pulse_dir, load_amps_file)))
    
    t_segs = np.array(load_pulse_times)
    amp_segs = np.array(load_pulse_amps)

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
freq_hist = []
cost_hist = []

In [5]:
# Run vqe, etc
vmax = np.vectorize(max)
vmin = np.vectorize(min)

hdf5_start_index = 0
start_segs = start_split_num
for i in range(max_segs):

    client_args = [chs, verbose, plot_opt_pulses, plot_pulse_viewer, time_start, time_stop, drive_freqs, hdf5_file_path]
    
    bounds = np.append(np.array(qscale), np.array(cscale))

    res = minimize(exp_g1_cost_func_xt, x0=np.append(amp_segs.flatten(), t_segs[:, 1:-1].flatten()), method='BFGS', args=client_args, options={"maxiter": ((i * maxiter_step) + maxiter_start)})
    
    sol = res.x
    
    # Update amps and times for next round
    opt_amps = sol[:2 * num_drives * (start_split_num)]
    
    opt_times = sol[2 * num_drives * start_split_num:]
    
    opt_result = res.fun
    
    # updates amplitudes and times with optimized values and reshape
    amp_segs = np.array(opt_amps)
    amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))
    
    t_segs = np.array(opt_times)
    t_segs = np.reshape(t_segs, (num_drives * 2, int(len(t_segs.flatten()) / (num_drives * 2))))
    
    print(f'================')
    print(f'num segs: {i + start_segs} ')
    print(f'opt_amps: {amp_segs}')
    print(f'opt_times: {t_segs}')
    print(f'opt_result: {opt_result}')

    # save values to history arrays
    time_hist.append(interp_time_wrapper(t_segs, time_start, time_stop))
    amp_hist.append(interp_amp_wrapper(amp_segs))
    cost_hist.append(opt_result)
    
    np.savez(r'run_data\\' + hdf5_name[:-3] + "-" + str(i) + ".npz", time=time_hist[-1], amp=amp_hist[-1], cost=cost_hist[-1])

    for i in range(2 * n_seg_jump):
        qscale.append((-init_qamp * qubit_amp_scale, init_qamp * qubit_amp_scale))
        cscale.append((-init_camp * cavity_amp_scale, init_camp * cavity_amp_scale))

    # split segments and return to start of loop
    if (i < max_segs - 1):
        for i in range(n_seg_jump):
            t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))




Previous cost: 0.1636709614



Previous cost: 0.148983075



Previous cost: 0.1179299226



Previous cost: 0.1708946459



Previous cost: 0.1517187891



Previous cost: 0.1500180619



Previous cost: 0.1583477342



Previous cost: 0.2034932349



Previous cost: 0.1139336002



Previous cost: 0.1435322454



Previous cost: 0.1382829742



Previous cost: 0.1401462821



Previous cost: 0.1386769828



Previous cost: 0.139879824



Previous cost: 0.1458334177



Previous cost: 0.1545437178



Previous cost: 0.1570766068



Previous cost: 0.1802229043



Previous cost: 0.160555541



Previous cost: 0.1742054949



Previous cost: 0.1400196364



Previous cost: 0.1519212559



Previous cost: 0.164536288



Previous cost: 0.1639093047



Previous cost: 0.1384438707



Previous cost: 0.1336244765



Previous cost: 0.1705805533



Previous cost: 0.1723362837



Previous cost: 0.1970078387



Previous cost: 0.1491101438



Previous cost: 0.1548748748



Previous cost: 0.1493360171



Previous 

ValueError: cannot reshape array of size 48 into shape (4,10)

In [6]:
print(sol)
print(len(sol))

[ 2.07746364e+02  2.08060695e+02 -2.08070951e+02 -2.08130072e+02
  2.08226777e+02  2.08566786e+02  1.59670198e+02 -2.08999990e+02
 -7.93683902e+02  8.15134448e+02  9.42000003e+02  8.81738037e+02
  8.87441005e+02 -8.87592788e+02 -8.87602783e+02  8.89516400e+02
  9.41999991e+02 -5.59306557e+02 -4.24000007e+02 -1.03237082e+01
  8.18562011e+01  1.67111955e+01  4.46500000e+03  1.97999996e+02
 -8.67549819e+01 -8.64817668e+01  8.47027122e+01  8.37851491e+01
  8.02655551e+01 -5.97691247e+01 -4.25663277e+01  5.92126552e+00
  4.33814620e+00  1.79104347e+03  3.75500000e+03 -4.34833188e+03
 -4.44184037e+03  4.44252044e+03  4.44274602e+03 -4.44307161e+03
 -4.44903687e+03 -2.85522354e+03  4.45868698e+03  4.46500000e+03
  5.84968123e-08  2.95244498e-06 -9.73327195e-07 -5.99575051e-06
 -1.28469578e-05 -4.61613506e-06  1.21753198e-05  1.44023641e-05
 -6.07335441e-06  3.64257255e-07  1.99554788e-06  2.56302056e-06
 -9.11726508e-07  1.37644997e-06  5.66757795e-06  5.09030343e-06
 -9.87791440e-06 -3.51268